## Decision Tree & Random Forest Models

Rather than handling the data as a series of time series, in these models I create basic features from the time series data for baseline performance.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import duckdb

In [30]:
# Create the rowwise sum of n days of observations for the specified range of columns; this range includes BOTH the start and end columns.
def n_day_sum(df, n=0, start_col=0, end_col=0, name_prefix=''):
    try:
        if (end_col - (start_col-1))%n != 0:
            raise ValueError('Column range must be evenly divisible by n.')
    except ValueError as e:
        print(f'{e}; DataFrame Unchanged; invalid column range. Start & end columns are included in range.')
        return df

    col_index = 0
    col_range = end_col - (start_col-1)
    cols_added = 0
    while col_index < col_range:
        sum_columns = df.iloc[:, start_col+col_index-1:start_col+col_index+n-1]
        sum = sum_columns.sum(axis=1)
        df[f'{name_prefix}nSum{n}_{cols_added}'] = sum
        col_index += n
        cols_added += 1

    return df

In [37]:
# Create the rowwise average of n days of observations for the specified range of columns; this range includes BOTH the start and end columns.
def n_day_mean(df, n=0, start_col=0, end_col=0, name_prefix=''):
    try:
        if (end_col - (start_col-1))%n != 0:
            raise ValueError('Column range must be evenly divisible by n.')
    except ValueError as e:
        print(f'{e}; DataFrame Unchanged; invalid column range. Start & end columns are included in range.')

    df = n_day_sum(df, n=n, start_col=start_col, end_col=end_col, name_prefix=name_prefix)
    
    cols_avgd = 0
    num_sums = (end_col - (start_col-1))/n
    while cols_avgd < num_sums:
        df[f'{name_prefix}nSum{n}_{cols_avgd}'] = df[f'{name_prefix}nSum{n}_{cols_avgd}']/n
        df.rename(columns={f'{name_prefix}nSum{n}_{cols_avgd}': f'{name_prefix}nMean{n}_{cols_avgd}'})
        cols_avgd += 1

    return df

In [12]:
# Create the global yearly average & attach it to the dataframe.
def yearly_means(df):
    n_day_mean(df, n=240, start_col=6, end_col=245)
    query = duckdb.query('SELECT year, AVG(nMean240_0) FROM pr_m GROUP BY year ORDER BY year;').to_df()
    df.merge(query, on='year')
    return df

In [38]:
pr_m = pd.read_parquet('/Users/james/FutureCrop_Kaggle/Data/pr_maize_train.parquet')

In [39]:
pr_m = n_day_mean(pr_m, n=10, start_col=6, end_col=245)
pr_m.head()

KeyboardInterrupt: 

In [26]:
pr_m.head()

,crop,year,lon,lat,variable,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,maize,381.0,-122.25,48.25,pr,0.000227,0.000149,0.000017,0.000000,0.000021,0.000031,0.000000,0.000135,0.000049,0.000016,0.000059,0.000021,0.000126,0.000112,0.000025,0.000138,0.000139,0.000000,0.000000,0.000075,0.0,0.000025,0.000063,0.000124,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000005,0.000000,0.000083,0.000152,0.000224,0.000018,0.000006,0.000000,0.0,0.000000,0.000003,0.000011,0.000076,0.000184,0.000009,0.000000,0.000000,0.000037,0.000122,0.000064,0.000013,0.000012,0.000125,0.000018,0.000000,0.0,0.0,0.000015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000001,0.000002,0.000003,0.000000,0.000009,0.000013,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000131,0.000308,0.000148,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000004,0.000029,0.0,0.000268,0.0,0.000070,0.0,0.0,0.000002,0.000000,0.000009,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000006,0.000017,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000003,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000170,0.000063,0.000004,0.000001,0.000002,0.000000,0.0,0.0,0.000000,0.000055,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000001,0.000000,0.000174,0.000005,0.000083,0.000000,0.000029,0.000266,0.000026,0.000002,0.000077,0.000008,0.000064,0.000114,0.000375,0.000198,0.000094,0.000030,0.000003,0.000000,0.000000,0.000000,0.000052,0.000075,0.000109,0.000122,0.000053,0.000457,0.000086,0.000070,0.000005,0.000035,0.000045,0.000074,0.000010,0.000073,0.0,0.000014,0.000081,0.000130,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000008,0.000057,0.000116,0.000436,0.000205,0.000103,0.000176,0.000073,0.000444,0.000279,0.000053,0.000169,0.000131,0.000122,0.000011,0.000396,0.000183,0.000146,0.000000,0.000069,0.000131
1,maize,381.0,-122.25,48.75,pr,0.000242,0.000128,0.000000,0.000000,0.000000,0.000035,0.000004,0.000084,0.000048,0.000007,0.000048,0.000010,0.000082,0.000093,0.000018,0.000108,0.000175,0.000017,0.000025,0.000167,0.0,0.000046,0.000054,0.000178,0.000001,0.000002,0.0,0.000000,0.000003,0.0,0.0,0.000000,0.000000,0.000000,0.000006,0.000203,0.000294,0.000299,0.000012,0.000000,0.000000,0.0,0.000010,0.000031,0.000023,0.000046,0.000181,0.000019,0.000002,0.000000,0.000035,0.000070,0.000145,0.000020,0.000000,0.000090,0.000038,0.000000,0.0,0.0,0.000007,0.000000,0.000000,0.000010,0.000000,0.000000,0.000012,0.000007,0.000005,0.000000,0.000035,0.000026,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000120,0.000406,0.000185,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000010,0.000209,0.0,0.000216,0.0,0.000034,0.0,0.0,0.000001,0.000000,0.000008,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000031,0.000021,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000

In [ ]:
pr_m = yearly_means(pr_m)
pr_m.head()

In [32]:
pr_m = n_day_sum(pr_m, n=10, start_col=6, end_col=245)

In [33]:
pr_m.head()

,crop,year,lon,lat,variable,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,nSum10_0,nSum10_1,nSum10_2,nSum10_3,nSum10_4,nSum10_5,nSum10_6,nSum10_7,nSum10_8,nSum10_9,nSum10_10,nSum10_11,nSum10_12,nSum10_13,nSum10_14,nSum10_15,nSum10_16,nSum10_17,nSum10_18,nSum10_19,nSum10_20,nSum10_21,nSum10_22,nSum10_23
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,maize,381.0,-122.25,48.25,pr,0.000227,0.000149,0.000017,0.000000,0.000021,0.000031,0.000000,0.000135,0.000049,0.000016,0.000059,0.000021,0.000126,0.000112,0.000025,0.000138,0.000139,0.000000,0.000000,0.000075,0.0,0.000025,0.000063,0.000124,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000005,0.000000,0.000083,0.000152,0.000224,0.000018,0.000006,0.000000,0.0,0.000000,0.000003,0.000011,0.000076,0.000184,0.000009,0.000000,0.000000,0.000037,0.000122,0.000064,0.000013,0.000012,0.000125,0.000018,0.000000,0.0,0.0,0.000015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000001,0.000002,0.000003,0.000000,0.000009,0.000013,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000131,0.000308,0.000148,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000004,0.000029,0.0,0.000268,0.0,0.000070,0.0,0.0,0.000002,0.000000,0.000009,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000006,0.000017,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000003,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000170,0.000063,0.000004,0.000001,0.000002,0.000000,0.0,0.0,0.000000,0.000055,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000001,0.000000,0.000174,0.000005,0.000083,0.000000,0.000029,0.000266,0.000026,0.000002,0.000077,0.000008,0.000064,0.000114,0.000375,0.000198,0.000094,0.000030,0.000003,0.000000,0.000000,0.000000,0.000052,0.000075,0.000109,0.000122,0.000053,0.000457,0.000086,0.000070,0.000005,0.000035,0.000045,0.000074,0.000010,0.000073,0.0,0.000014,0.000081,0.000130,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000008,0.000057,0.000116,0.000436,0.000205,0.000103,0.000176,0.000073,0.000444,0.000279,0.000053,0.000169,0.000131,0.000122,0.000011,0.000396,0.000183,0.000146,0.000000,0.000069,0.000131,0.000645,0.000694,0.000212,0.000488,0.000283,0.000390,0.000021,0.000022,0.000588,0.000372,0.000009,0.000023,0.000000,0.000003,0.000170,0.000125,0.000176,0.000561,0.000866,0.001058,0.000382,0.000008,0.001943,0.001357
1,maize,381.0,-122.25,48.75,pr,0.000242,0.000128,0.000000,0.000000,0.000000,0.000035,0.000004,0.000084,0.000048,0.000007,0.000048,0.000010,0.000082,0.000093,0.000018,0.000108,0.000175,0.000017,0.000025,0.000167,0.0,0.000046,0.000054,0.000178,0.000001,0.000002,0.0,0.000000,0.000003,0.0,0.0,0.000000,0.000000,0.000000,0.000006,0.000203,0.000294,0.000299,0.000012,0.000000,0.000000,0.0,0.000010,0.000031,0.000023,0.000046,0.000181,0.000019,0.000002,0.000000,0.000035,0.000070,0.000145,0.000020,0.000000,0.000090,0.000038,0.00